In [361]:
# import statements
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import sklearn

In [362]:
# read in data
train_df = pd.read_csv("data/train.csv") 
test_df = pd.read_csv("data/test.csv")

In [363]:
# preview the datatest_df.head()
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [364]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [365]:
# see info about data
train_df.info()
print("----------------------------")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
----------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare

In [366]:
# a naive solution, ignoring data cleaning and other useful relationships

# clean data
train_df = train_df.drop(['Ticket','Cabin', 'Embarked', 'Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Ticket','Cabin', 'Embarked', 'Name'], axis=1)
train_df = train_df.dropna()


In [367]:
# set nan for age to the mean of the age column
train_df['Age'] = train_df.groupby(['Pclass'])['Age'].transform(lambda x: x.fillna(x.mean()))
test_df['Age'] = test_df.groupby(['Pclass'])['Age'].transform(lambda x: x.fillna(x.mean()))

# set nan fare to the mean of the fare column
train_df['Fare'] = train_df.groupby(['Pclass'])['Fare'].transform(lambda x: x.fillna(x.mean()))
test_df['Fare'] = test_df.groupby(['Pclass'])['Fare'].transform(lambda x: x.fillna(x.mean()))

# make nan for sex male since men are more likely to not
# survive, and most dont survive
train_df['Sex'] = train_df.groupby(['Pclass'])['Sex'].transform(lambda x: x.fillna('male'))
test_df['Sex'] = test_df.groupby(['Pclass'])['Sex'].transform(lambda x: x.fillna('male'))

#train_df['Age'] = train_df['Age'].astype(int)
#test_df['Age'] = test_df['Age'].astype(int)

#train_df.loc[train_df['Age'] <= 16, 'Age'] = 0
#train_df.loc[(train_df['Age'] > 16) & (train_df['Age'] <= 32), 'Age'] = 1
#train_df.loc[(train_df['Age'] > 32) & (train_df['Age'] <= 48), 'Age'] = 2
#train_df.loc[(train_df['Age'] > 48) & (train_df['Age'] <= 64), 'Age'] = 3
#train_df.loc[(train_df['Age'] > 64), 'Age'] = 4

#test_df.loc[ test_df['Age'] <= 16, 'Age'] = 0
#test_df.loc[(test_df['Age'] > 16) & (test_df['Age'] <= 32), 'Age'] = 1
#test_df.loc[(test_df['Age'] > 32) & (test_df['Age'] <= 48), 'Age'] = 2
#test_df.loc[(test_df['Age'] > 48) & (test_df['Age'] <= 64), 'Age'] = 3
#test_df.loc[(test_df['Age'] > 64), 'Age'] = 4



In [368]:
# make everything ints
train_df['Age'] = train_df['Age'].astype(int)
test_df['Age'] = test_df['Age'].astype(int)
train_df['Fare'] = train_df['Fare'].astype(int)
# test_df['Fare'] = test_df['Fare'].astype(int)
# set men to 1 and female to 0
train_df['Sex'] = train_df['Sex'].map(lambda s : 1 if s == 'male' else 0)
test_df['Sex'] = test_df['Sex'].map(lambda s : 1 if s == 'male' else 0)
test_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,3,1,34,0,0,7.8292
1,893,3,0,47,1,0,7.0000
2,894,2,1,62,0,0,9.6875
3,895,3,1,27,0,0,8.6625
4,896,3,0,22,1,1,12.2875


In [369]:
# define data
X_train = train_df.drop("Survived",axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId",axis=1).copy()

In [370]:
# test random forest classifier
random_forest = RandomForestClassifier(n_estimators=100)
#random_forest = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=10, max_features='sqrt', min_samples_split=5)

random_forest.fit(X_train, Y_train)

Y_pred = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)

0.96358543417366949

In [371]:
# gradient boost
#gradient_boost = GradientBoostingClassifier(n_estimators=100,loss='exponential',max_features='log2')
gradient_boost = GradientBoostingClassifier(n_estimators=100)
gradient_boost.fit(X_train, Y_train)

Y_pred_2 = gradient_boost.predict(X_test)

gradient_boost.score(X_train, Y_train)

0.89775910364145661

In [372]:
# extra tree
#extra_tree = ExtraTreesClassifier(n_estimators=100,criterion='gini',max_depth=10,max_features='log2',min_samples_split=10)
extra_tree = ExtraTreesClassifier(n_estimators=100)
extra_tree.fit(X_train, Y_train)

Y_pred_3 = extra_tree.predict(X_test)

extra_tree.score(X_train, Y_train)

0.96358543417366949

In [373]:
# kNearest Neighbors
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 7,weights='distance')
knn.fit(X_train, Y_train)
Y_pred_4 = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

96.36

In [374]:
# Ada boost
ada_boost = AdaBoostClassifier(n_estimators=100,algorithm='SAMME')

ada_boost.fit(X_train, Y_train)

Y_pred_5 = ada_boost.predict(X_test)

ada_boost.score(X_train, Y_train)

0.82072829131652658

In [384]:
# Gaussian Naive Bayes

gaussian = GaussianNB()

gaussian.fit(X_train, Y_train)

Y_pred_6 = gaussian.predict(X_test)

gaussian.score(X_train, Y_train)

0.78991596638655459

In [385]:
# set which alg you want to predict
Y_pred = Y_pred_6

In [386]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('titanic.csv', index=False)